In [4]:
import math
import numpy as np
from tic_tac_toe import TicTacToeEnv
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
def minimax(state, action, turn):
    state_, reward, done = env.move(state, action, 1)
    if done:
        return reward

    possible_actions = env.possible_actions()
    
    if turn == 1:
        max_value = -np.inf

    if turn == 0:
        max_value = np.inf

    for i in possible_actions:
        value = minimax(state_, i, abs(1-turn))
        if turn == 1:
            max_value = max(max_value, value)
        else:
            max_value = min(max_value, value)

    return max_value

def search(state):
    board = state
    max_value = -np.inf
    max_action = None
    for i in range(9):
        value = minimax(board, i, 0)
        if value > max_value:
            max_action = i
            max_value = value

    return max_action

In [73]:
class Node:
    def __init__(self, parent, possible_actions, turn, player):
        self.state = None
        self.parent = parent
        self.player = player
        self.value = 0
        self.visits = 0
        self.children = [None for i in range(9)]
        self.possible_actions = possible_actions
        self.fully_expanded = False
        self.updated_node = True
        self.turn = turn
        self.terminal = False

    @property
    def depth(self):
        count = 0
        node = self
        while not node.parent is None:
            count += 1
            node = node.parent

        return count

    def expand(self, env, turn):
        if self.fully_expanded or not self.updated_node:
            print('the node is already fully expanded or its not updated')

        state_1 = self.state.copy()
        playa = self.player

        for action in self.possible_actions:
            self.state = state_1
            next_state, reward, done = env.move(self.state, action, self.player, turn)
            next_possible_actions = self.possible_actions.copy()
            next_possible_actions.remove(action)
            
            node = Node(self, next_possible_actions, self.turn, abs(1-playa))
            if done or len(next_possible_actions) == 0:
                node.terminal = True
            node.state = next_state
            node.updated_node = True
            self.children[action] = node

        self.player = playa
        flag = False
        for i in self.possible_actions:
            if self.children[i] is None:
                flag = True
        if not flag:
            self.fully_expanded = True

    def choose_node(self, scalar):
        max_ucb = -np.inf
        max_node = []
        ucbs = []

        for action in self.possible_actions:
            child = self.children[action]
            if child.visits > 0:
                ucb = child.value / child.visits + scalar * math.sqrt(math.log(self.visits / child.visits))
            else:
                ucb = np.inf
            ucbs.append(ucb)
            if ucb > max_ucb:
                max_ucb = ucb
                max_node.append(child)

        return np.random.choice(max_node)

    def choose_action(self, scalar):
        max_ucb = -np.inf
        max_action = []

        for action in self.possible_actions:
            child = self.children[action]
            if child.visits > 0:
                ucb = child.value / child.visits + scalar * math.sqrt(math.log(self.visits / child.visits))
            else:
                ucb = np.inf
            if ucb > max_ucb:
                max_ucb = ucb
                max_action.append(action)

        return np.random.choice(max_action)

    def backpropogate(self, value):
        node = self
        while node is not None:
            node.visits += 1
            node.value += value
            node = node.parent


class MCTS:
    def __init__(self, env):
        self.env = env

    def search(self, init_state, turn):
        possible_actions = self.env.possible_actions()
        start_node = Node(None, possible_actions, turn, turn)
        start_node.state = init_state

        for alo in range(1000):
            node = start_node

            # traverse
            if node.fully_expanded:
                while node.fully_expanded:
                    node = node.choose_node(1/math.sqrt(2))


            node_terminaled = False
            if node.terminal:
                #print(node.state)
                #print('terminaled')
                action = node.parent.children.index(node)
                _, reward, _  = self.env.move(node.parent.state, action, node.parent.player, turn)
                node.backpropogate(reward)
                node_terminaled = True
                continue

            # expand
            node.expand(self.env, turn)
            node = node.choose_node(2)

            if node.terminal and not node_terminaled:
                #print('terminaled')
                action = node.parent.children.index(node)
                _, reward, _ = self.env.move(node.parent.state, action, node.parent.player, turn)
                node.backpropogate(reward)
                continue

            # rollout
            player = node.player
            done = False
            possible_actions = node.possible_actions.copy()
            state = node.state
            while not done:
                if len(possible_actions) > 0:
                    #print(player, 'player')
                    action = np.random.choice(possible_actions)
                    state, reward, done = self.env.move(
                        state,
                        action,
                        player,
                        turn
                    )
                    #env.print_board(state)
                    #if done:
                        #print(reward)
                    possible_actions.remove(action)

                else:
                    done = True
                    reward = 0

                if done:
                    #print('backpropp')
                    #env.print_board(state)
                    node.backpropogate(reward)

                player = abs(1-player)


        print(start_node.value, start_node.visits)
        return start_node.choose_action(1/math.sqrt(2))


In [79]:
env = TicTacToeEnv()
agent = MCTS(env)
state = env.reset()

In [82]:
action = agent.search(state, 1)
state_, _, _, _ = env.step(action)
env.print_board()
action = agent.search(state_, 0)
state_, _, _, _ = env.step(action)
state = state_
env.print_board()

603 1000
alo
1
1 won!
 0 1 # 
 # 1 # 
 0 1 # 

0 1000
alo
0
0 won!
 0 1 # 
 0 1 # 
 0 1 # 



In [12]:
np.random.choice([])

ValueError: 'a' cannot be empty unless no samples are taken

In [83]:
import p5

In [84]:
from p5 import *

In [86]:
def setup():
    size(600, 600)
    
def draw():
    background(0)

In [88]:
run()

  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/kae/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kae/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/home/kae/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 612, in start
    self.io_loop.start()
  File "/home/kae/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
    self._run_once()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 1859, in _run_once
    handle._run()
  File "/usr/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.

AttributeError: module 'p5.core.p5' has no attribute 'exit'